In [1]:
# Import dependencies
import pandas as pd
%pip install psycopg2
import psycopg2

Note: you may need to restart the kernel to use updated packages.


In [63]:
# Import csv files into dataframes
emissions_df = pd.read_csv('../Resources/US_Historical_Clean_All_Gases.csv', index_col = 0)
gdp_df = pd.read_csv('../Resources/US_GDPbyState_Cleaned.csv', index_col = 0)
pop_df = pd.read_csv('../Resources/US_PopbyState_Cleaned.csv', index_col = 0)

In [77]:
#Create emissions_no_sector_df
emissions_no_sector_df = emissions_df.drop('Sector', axis = 1)
emissions_no_sector_df

,Country,State,Year,All GHG,CH4,CO2,N2O,F-Gas
0,US,AL,1990,7.451525,0.000000,0.000000,0.000000,0.000000
1,US,AL,1991,7.277622,3.283175,0.000000,4.168350,0.515225
2,US,AL,1992,7.312519,3.205741,0.022604,4.071881,0.499681
3,US,AL,1993,7.677440,3.249096,0.022895,4.063423,0.525109
4,US,AL,1994,7.338171,3.293162,0.020906,4.384278,0.584388
...,...,...,...,...,...,...,...,...
18091,US,WY,2014,0.449731,0.000000,0.000000,0.000000,0.000000
18092,US,WY,2015,0.456765,0.000000,0.000000,0.000000,0.000000
18093,US,WY,2016,0.463380,0.000000,0.000000,0.000000,0.000000
18094,US,WY,2017,0.469695,0.000000,0.000000,0.000000,0.000000


In [65]:
gdp_df

,State,Abbrev,Year,GDP
0,United States,USA,1997,3.286530e+12
1,United States,USA,1998,3.384010e+12
2,United States,USA,1999,3.592860e+12
3,United States,USA,2000,3.733600e+12
4,United States,USA,2001,3.866600e+12
...,...,...,...,...
1243,Wyoming,WY,2016,1.516490e+10
1244,Wyoming,WY,2017,1.597636e+10
1245,Wyoming,WY,2018,1.712228e+10
1246,Wyoming,WY,2019,1.697884e+10


In [66]:
# Remove decimal from Year in pop_df
pop_df['Year'] = pop_df['Year'].astype(int)

In [67]:
# Add total US Population to Pop DF
total_pop = pop_df.groupby(pop_df['Year']).sum()['Population']
total_pop = pd.DataFrame(total_pop).reset_index()
total_pop = total_pop.reindex(columns = ['State', 'Year', 'Population'])
pop_df_with_total = pop_df.append(total_pop)
pop_df_with_total.fillna('United States', inplace = True)
pop_df_with_total = pop_df_with_total.reset_index(drop = True)
pop_df_with_total

,State,Year,Population
0,Alabama,1990,4050055
1,Alabama,1991,4099156
2,Alabama,1992,4154014
3,Alabama,1993,4214202
4,Alabama,1994,4260229
...,...,...,...
1607,United States,2016,323071755
1608,United States,2017,325122128
1609,United States,2018,326838199
1610,United States,2019,328329953
